In [3]:
#imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import PhaseGate
from qiskit_aer import AerSimulator
from collections import Counter
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
!pip install qiskit qiskit-aer qiskit[visualization] matplotlib pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.3 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=64dbe83418dd7508f03824e08e404213def69ed87ff3adaf2517764a31bfa87a
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [14]:
def iqpe(phase, t=7, shots=1024):
  backend = AerSimulator()
  bits = []

  for k in range(1, t + 1):
      power = 2 ** (t - k)

      qr = QuantumRegister(2)
      cr = ClassicalRegister(1)
      qc = QuantumCircuit(qr, cr)

      qc.x(qr[1])

      qc.h(qr[0])

      angle = 2 * np.pi * phase * power
      qc.append(PhaseGate(angle).control(1), [qr[0], qr[1]])

      if bits:
          feedback_angle = -2 * np.pi * sum(
              bits[j] / (2 ** (j + 1)) for j in range(len(bits))
          )
          qc.p(feedback_angle, qr[0])

      qc.h(qr[0])
      qc.measure(qr[0], cr[0])

      counts = backend.run(qc, shots=shots).result().get_counts()
      bit = int(max(counts, key=counts.get))
      bits.append(bit)

  phi_est = 0
  for i, b in enumerate(reversed(bits)):
      phi_est += b / (2 ** (i + 1))
  return phi_est, bits

In [25]:
phi = float(input("Enter the phase (0 ≤ φ < 1): "))

estimate, bitstring = iqpe(phi, t=7, shots=1024)

print("\nResults")
print(f"Estimated phase: {estimate:.8f}")
print(f"Original phase:  {phi:.8f}")

Enter the phase (0 ≤ φ < 1): 0.7890987

Results
Estimated phase: 0.63281250
Original phase:  0.78909870
